# Setup Code

### Drive Setup

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### DLC Setup
Code created to automatically crash the notebook to reload the dependencies that were imported. Requires manual comment of os.kill line of code after the first run

In [0]:
# Download and installation
%cd /content
!git clone -l -s git://github.com/AlexEMG/DeepLabCut.git cloned-DLC-repo
%cd cloned-DLC-repo

from IPython.display import clear_output
# !pip install deeplabcut
clear_output()

#### Setup.py write

In [0]:
%%writefile setup.py
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
DeepLabCut2.0 Toolbox (deeplabcut.org)
© A. & M. Mathis Labs
https://github.com/AlexEMG/DeepLabCut

Please see AUTHORS for contributors.
https://github.com/AlexEMG/DeepLabCut/blob/master/AUTHORS
Licensed under GNU Lesser General Public License v3.0
"""

import setuptools

with open("README.md", "r") as fh:
    long_description = fh.read()

setuptools.setup(
    name="deeplabcut",
    version="2.0.9",
    author="A. & M. Mathis Labs",
    author_email="alexander.mathis@bethgelab.org",
    description="Markerless pose-estimation of user-defined features with deep learning",
    long_description=long_description,
    long_description_content_type="text/markdown",
    url="https://github.com/AlexEMG/DeepLabCut",
    install_requires=['certifi','chardet~=3.0.4','click','easydict~=1.7',
                      'gast==0.2.2','h5py~=2.7','imageio~=2.3.0','intel-openmp',
                      'ipython~=6.0.0','ipython-genutils~=0.2.0',
                      'matplotlib~=3.0.3','moviepy~=0.2.3.5','numpy~=1.14.5','opencv-python~=3.4',
                      'pandas>=0.21.0','patsy','python-dateutil~=2.7.3','pyyaml>=5.1','requests',
                      'ruamel.yaml~=0.15','setuptools','scikit-image~=0.14.0','scikit-learn~=0.19.2',
                      'scipy~=1.1.0','statsmodels~=0.9.0','tables',
                      'tensorpack~=0.9.7.1',
                      'tqdm>4.29','wheel~=0.31.1'],
    scripts=['deeplabcut/pose_estimation_tensorflow/models/pretrained/download.sh'],
    packages=setuptools.find_packages(),
    data_files=[('deeplabcut',['deeplabcut/pose_cfg.yaml','deeplabcut/pose_estimation_tensorflow/models/pretrained/pretrained_model_urls.yaml'])],
    include_package_data=True,
    classifiers=(
        "Programming Language :: Python :: 3",
        "License :: OSI Approved :: GNU Lesser General Public License v3 (LGPLv3)",
        "Operating System :: OS Independent",
    ),
    entry_points="""[console_scripts]
            dlc=dlc:main""",
)

#https://stackoverflow.com/questions/39590187/in-requirements-txt-what-does-tilde-equals-mean

Overwriting setup.py


### Remaining Setup

In [0]:
%cd /content
!pip install -e cloned-DLC-repo
clear_output()

import os
# os.kill(os.getpid(), 9)     # Comment this line out after first run


In [0]:
# Environment setup 

# GUIs don't work on the cloud, so we will supress wxPython: 
%cd /content/cloned-DLC-repo
os.environ["DLClight"]="True"
os.environ["Colab"]="True"

import deeplabcut

# Create a path variable that links to the config file:
from pathlib import Path
path_config_file = '/content/drive/Shared drives/Final Year Project/Datasets/Cheetah-AnChi-2019-04-02/config_colab.yaml'
path_pose_config_file = '/content/drive/Shared drives/Final Year Project/Datasets/Cheetah-AnChi-2019-04-02/dlc-models/iteration-4/CheetahApr2-trainset95shuffle1/train/pose_cfg_colab.yaml'
# Options: Batch8; ImgRework; LR4Step; SetScales1; DoubleHourglass; ImgReworkDS; SetScales2; DoubleHourglassCLR; InterSup; QDoubleHourglass; tInterSup; FsIsTripleHourglass; LR1Ext; ScmapRework; TripleHourglass
model_version =  'FsIsTripleHourglass'
eval_interm = True
eval_hourglass_stack = 3
eval_full_scale = True
eval_snap_index = -1
path_extension = str(Path(path_pose_config_file).parents[4] / 'extension-models' / Path(path_pose_config_file).parents[2].stem / Path(path_pose_config_file).parents[1].stem / Path(path_pose_config_file).parents[0].stem / model_version)
plotting = False 

/content/cloned-DLC-repo
Project loaded in colab-mode. Apparently Colab has trouble loading statsmodels, so the smoothing & outlier frame extraction is disabled. Sorry!
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

DLC loaded in light mode; you cannot use the labeling GUI!


# Evaluation

## Model Definition 

### Bottleneck Module Definition



In [0]:
from tensorflow.contrib import layers
from tensorflow.contrib.framework.python.ops import add_arg_scope
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.contrib.slim.python.slim.nets import resnet_utils
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import variable_scope

@add_arg_scope
def bottleneck(inputs,
               depth,
               depth_bottleneck,
               stride,
               rate=1,
               outputs_collections=None,
               scope=None):
  """Bottleneck residual unit variant with BN after convolutions.
  This is the original residual unit proposed in [1]. See Fig. 1(a) of [2] for
  its definition. Note that we use here the bottleneck variant which has an
  extra bottleneck layer.
  When putting together two consecutive ResNet blocks that use this unit, one
  should use stride = 2 in the last unit of the first block.
  Args:
    inputs: A tensor of size [batch, height, width, channels].
    depth: The depth of the ResNet unit output.
    depth_bottleneck: The depth of the bottleneck layers.
    stride: The ResNet unit's stride. Determines the amount of downsampling of
      the units output compared to its input.
    rate: An integer, rate for atrous convolution.
    outputs_collections: Collection to add the ResNet unit output.
    scope: Optional variable_scope.
  Returns:
    The ResNet unit's output.
  """
  with variable_scope.variable_scope(scope, 'bottleneck_v1', [inputs]) as sc:
    depth_in = utils.last_dimension(inputs.get_shape(), min_rank=4)
    if depth == depth_in:   #PSE
      shortcut = resnet_utils.subsample(inputs, stride, 'shortcut')
    else:
        shortcut = layers.conv2d(
            inputs,
            depth, [1, 1],
            stride=stride,
            activation_fn=None,
            scope='shortcut')
    residual = layers.conv2d(
        inputs, depth_bottleneck, [1, 1], stride=1, scope='conv1')
    residual = resnet_utils.conv2d_same(
        residual, depth_bottleneck, 3, stride, rate=rate, scope='conv2')
    residual = layers.conv2d(
        residual, depth, [1, 1], stride=1, activation_fn=None, scope='conv3')

    output = nn_ops.relu(shortcut + residual)

    return utils.collect_named_outputs(outputs_collections, sc.name, output)

### Regnet Definition and Help functions

In [0]:
# Helper functions for regnet
from tensorflow.contrib.layers.python.layers import layers as layers_lib

def regnet_block_1(inputs,scope='regnet_block_1_'):
  net = layers.conv2d(
      inputs, 64, 7, stride=2,padding='SAME', scope=scope+'conv1')
  net = layers_lib.max_pool2d(
      net, [2, 2], stride=2, padding='SAME', scope=scope+'pool1')
  
  return net

def regnet_block_2(inputs,
               depth_bottleneck,
               depth=256,
               stride=1,
               rate=1,
               scope=None):
  
  # Three bottleneck modules
  # Scope passed to function should include instance number
  if scope == None:
    net = bottleneck(
        inputs,depth,depth_bottleneck,stride,rate)
    net = bottleneck(
        net,depth,depth_bottleneck,stride,rate)
    net = bottleneck(
        net,depth,depth_bottleneck,stride,rate)
  else:
    net = bottleneck(
        inputs,depth,depth_bottleneck,stride,rate,scope=scope+'_1')
    net = bottleneck(
        net,depth,depth_bottleneck,stride,rate,scope=scope+'_2')
    net = bottleneck(
        net,depth,depth_bottleneck,stride,rate,scope=scope+'_3')

  return  net

def regnet_block_3(inputs,
               depth_bottleneck,
               depth=256,
               stride=1,
               rate=1,
               scope=None):
  
  # Three bottleneck modules preceded by a maxpooling layer
  # Scope passed to function should include instance number

  if scope == None:
    net = layers.max_pool2d(
        inputs, [2, 2], stride=2, padding='SAME')
    net = bottleneck(
        net,depth,depth_bottleneck,stride,rate)
    net = bottleneck(
        net,depth,depth_bottleneck,stride,rate)
    net = bottleneck(
        net,depth,depth_bottleneck,stride,rate)     
  else:
    net = layers.max_pool2d(
        inputs, [2, 2], stride=2, padding='SAME', scope=scope+'pool')
    net = bottleneck(
        net,depth,depth_bottleneck,stride,rate,scope=scope+'_1')
    net = bottleneck(
        net,depth,depth_bottleneck,stride,rate,scope=scope+'_2')
    net = bottleneck(
        net,depth,depth_bottleneck,stride,rate,scope=scope+'_3')    

  return net

def regnet_block_4(inputs,
               depth_bottleneck,
               scope=None):
  
  # Helper function to pass straight to regnet_block_2
  # Assists in scope handling
  net = regnet_block_2(
      inputs, depth_bottleneck=depth_bottleneck, scope=scope)

  return net

def regnet_block_5(inputs,
               depth,
               scope=None):
  
  net = layers.conv2d_transpose(
      inputs, depth, [2, 2], stride=2, padding='SAME', scope=scope)   

  return net

def regnet_block_6(inputs,
               depth,
               scope=None):

  net = layers.conv2d(
        inputs, depth, [1, 1], stride=1, activation_fn=None, normalizer_fn=None, scope=scope, padding='SAME')   #toMod, activation and norm functions?
  
  return net

def regnet_block_7(inputs,
               num_outputs,
               scope=None):
  
  net = layers.conv2d(                                                                              
        inputs, num_outputs, [1, 1], stride=1, activation_fn=None, normalizer_fn=None, scope=scope, padding='SAME')   #toMod, activation and norm functions?

  return net

def regnet_block_8(inputs,
               num_outputs,
               scope=None):

  net = layers.conv2d_transpose(
    inputs, num_outputs, [2, 2], stride=2, scope=scope, padding='SAME')
  
  return net  


In [0]:
# Hourglass construction helper function
def hourglass(inputs, hg_depth, scope='HG_'):
  assert hg_depth >= 1
  hg_level = '_'+str(hg_depth)

  net = regnet_block_3(inputs, depth_bottleneck=128, scope=scope+'regnet_block_3'+hg_level)

  if hg_depth == 1: 
    shortcut = regnet_block_5(net, depth=256, scope=scope+'regnet_block_5'+hg_level)
  else:
    shortcut = regnet_block_5(hourglass(net,hg_depth-1,scope=scope), depth=256, scope=scope+'regnet_block_5'+hg_level)

  residual = regnet_block_4(inputs, depth_bottleneck=128, scope=scope+'regnet_block_4'+hg_level)

  return residual + shortcut     


In [0]:
# Regnet function definition

# Eval Parameters
# eval_interm = False
# eval_hourglass_stack = 1
# eval_full_scale = False

def regnet(inputs, scmap_inputs, num_outputs):
  if model_version == 'SetScales2':
    inputs = regnet_block_1(inputs)  
    stacked_input = tf.concat([inputs,scmap_inputs],-1)

    net = regnet_block_2(stacked_input, depth_bottleneck=64, scope='regnet_block_2_1')
    net = hourglass(net, hg_depth=4)
    net = regnet_block_6(net, depth=512, scope='regnet_block_6_1')
    net = regnet_block_6(net, depth=512, scope='regnet_block_6_2')
    net = regnet_block_7(net, num_outputs=num_outputs, scope='regnet_block_7_1')  
    net = regnet_block_8(net, num_outputs=num_outputs, scope='regnet_block_8_1')  

    return net 
  else: 
    #Input and Root Blocks
    inputs = regnet_block_1(inputs)  
    stacked_input = tf.concat([inputs,scmap_inputs],-1)
    net = regnet_block_2(stacked_input, depth_bottleneck=64, scope='regnet_block_2_1')

    if eval_interm:
      hg_dict = {}

    #Stacked Hourglass Definition
    for hg_num in range(1,eval_hourglass_stack+1):
      hg_name = str(hg_num)
      net = hourglass(net, hg_depth=4,scope='HG{}_'.format(hg_name))

      #Intermediate Supervision Outputs
      if eval_interm and (hg_num != eval_hourglass_stack):
        hg_dict[hg_name] = regnet_block_6(net, depth=512, scope='hg{}_block_6_1'.format(hg_num))
        hg_dict[hg_name] = regnet_block_6(hg_dict[hg_name], depth=512, scope='hg{}_block_6_2'.format(hg_num))
        hg_dict[hg_name] = regnet_block_7(hg_dict[hg_name], num_outputs=num_outputs, scope='hg{}_block_7_1'.format(hg_num))  
        if eval_full_scale:
          hg_dict[hg_name] = regnet_block_8(hg_dict[hg_name], num_outputs=num_outputs, scope='hg{}_block_8_1'.format(hg_num))  

    #Finishing Blocks
    net = regnet_block_6(net, depth=512, scope='regnet_block_6_1')
    net = regnet_block_6(net, depth=512, scope='regnet_block_6_2')
    net = regnet_block_7(net, num_outputs=num_outputs, scope='regnet_block_7_1')  
    if eval_full_scale:
      net = regnet_block_8(net, num_outputs=num_outputs, scope='regnet_block_8_1')  #Comment out for half scale, uncomment for full scale

    # hg1_out = regnet_block_6(hg1_net, depth=512, scope='hg1_block_6_1')
    # hg1_out = regnet_block_6(hg1_out, depth=512, scope='hg1_block_6_2')
    # hg1_out = regnet_block_7(hg1_out, num_outputs=num_outputs, scope='hg1_block_7_1')  
    # # hg1_out = regnet_block_8(hg1_out, num_outputs=num_outputs, scope='hg1_block_8_1')  #Comment out for half scale, uncomment for full scale

    # hg2_out = regnet_block_6(hg2_net, depth=512, scope='hg2_block_6_1')
    # hg2_out = regnet_block_6(hg2_out, depth=512, scope='hg2_block_6_2')
    # hg2_out = regnet_block_7(hg2_out, num_outputs=num_outputs, scope='hg2_block_7_1')  
    # # hg2_out = regnet_block_8(hg2_out, num_outputs=num_outputs, scope='hg2_block_8_1')  #Comment out for half scale, uncomment for full scale

    if eval_interm:
      hg_outputs = []
      for i in range(eval_hourglass_stack-1):
        hg_outputs.append(hg_dict[str(i+1)])
      return net, hg_outputs
    else:
      return net


## Auxiliary Function Modifications

In [0]:
def GetModelFolder(trainFraction,shuffle,cfg):
    Task = cfg['Task']
    date = cfg['date']
    iterate = 'iteration-'+str(cfg['iteration'])
    return Path('extension-models/'+ iterate+'/'+Task + date + '-trainset' + str(int(trainFraction * 100)) + 'shuffle' + str(shuffle))

In [0]:
def GetScorerName(cfg,shuffle,trainFraction,trainingsiterations='unknown'):
    ''' Extract the scorer/network name for a particular shuffle, training fraction, etc. '''
    Task = cfg['Task']
    date = cfg['date']
    if trainingsiterations=='unknown':
        snapshotindex=cfg['snapshotindex']
        if cfg['snapshotindex'] == 'all':
            print("Changing snapshotindext to the last one -- plotting, videomaking, etc. should not be performed for all indices. For more selectivity enter the ordinal number of the snapshot you want (ie. 4 for the fifth) in the config file.")
            snapshotindex = -1
        else:
            snapshotindex=cfg['snapshotindex']

        modelfolder=os.path.join(cfg["project_path"],str(GetModelFolder(trainFraction,shuffle,cfg)),'train')
        Snapshots = np.array([fn.split('.')[0]for fn in os.listdir(modelfolder) if "index" in fn])
        increasing_indices = np.argsort([int(m.split('-')[1]) for m in Snapshots])
        Snapshots = Snapshots[increasing_indices]
        #dlc_cfg = read_config(os.path.join(modelfolder,'pose_cfg.yaml'))
        #dlc_cfg['init_weights'] = os.path.join(modelfolder , 'train', Snapshots[snapshotindex])
        SNP=Snapshots[snapshotindex]
        trainingsiterations = (SNP.split(os.sep)[-1]).split('-')[-1]

    scorer = 'DLC_Extension' + "_resnet" + str(cfg['resnet']) + "_" + Task + str(date) + 'shuffle' + str(shuffle) + '_' + str(trainingsiterations)
    return scorer

## PoseNet Class 

### pose_net function definitions
Functions of the same name adapted from DeeperCut pose_net.py

#### get_batch_spec

In [0]:
# get_batch_spec() function

def get_batch_spec(cfg):
    num_joints = cfg.num_joints
    batch_size = cfg.batch_size
    scoremap_and_locref_channel_multiplier = 3
    return {  
        Batch.inputs: [batch_size, None, None, 3],
        Batch.part_score_targets: [batch_size, None, None, num_joints],
        Batch.part_score_weights: [batch_size, None, None, num_joints],
        Batch.locref_targets: [batch_size, None, None, num_joints * 2],
        Batch.locref_mask: [batch_size, None, None, num_joints * 2],
        Batch.scmap_inputs: [batch_size, None, None, num_joints * scoremap_and_locref_channel_multiplier]
    }

#### prediction_layer

In [0]:
# prediction_layer() function
#toMod -> should scope arg be changed here? -> happy with their layer hyper-parameters?

import tensorflow.contrib.slim as slim

def prediction_layer(cfg, input, name, num_outputs):
    with slim.arg_scope([slim.conv2d, slim.conv2d_transpose], padding='SAME',
                        activation_fn=None, normalizer_fn=None,
                        weights_regularizer=slim.l2_regularizer(cfg.weight_decay)):
        with TF.variable_scope(name):
            pred = layers.conv2d_transpose(input, num_outputs,            # changed slim to layers
                                         kernel_size=[2, 2], stride=2,    #adapted kernal stride
                                         scope='regnet_pred_layers', padding='SAME')    #inserted padding for sanity check
            return pred

#### PoseNet Class Definition

In [0]:
# PoseNet Class
# Mod for interm sup (also to throw away hg_output for eval)

from tensorflow.contrib.slim.python.slim.nets.resnet_utils import resnet_arg_scope

class PoseNet:
    def __init__(self, dlc_cfg):
        self.dlc_cfg = dlc_cfg

    def extract_features(self, inputs, scmap_inputs):
        mean = tf.constant(self.dlc_cfg.mean_pixel,
                           dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean') 
        im_centered = inputs - mean             #toMod, PSE, -> what is this doing? What should it be? Should I do the same with scmap_inputs

        # The next part of the code depends upon which tensorflow version you have.
        vers = tf.__version__
        vers = vers.split(".") #Updated based on https://github.com/AlexEMG/DeepLabCut/issues/44
        if int(vers[0])==1 and int(vers[1])<4: #check if lower than version 1.4.
            with slim.arg_scope(resnet_arg_scope(False)):
              if eval_interm:
                net = regnet(im_centered, scmap_inputs, self.dlc_cfg.num_joints)[0]
              else:
                net = regnet(im_centered, scmap_inputs, self.dlc_cfg.num_joints)
        else:
            with slim.arg_scope(resnet_arg_scope()):
              if eval_interm:
                net = regnet(im_centered, scmap_inputs, self.dlc_cfg.num_joints)[0]
              else:
                net = regnet(im_centered, scmap_inputs, self.dlc_cfg.num_joints)
        return net

    def prediction_layers(self, features, reuse=None):
        dlc_cfg = self.dlc_cfg

        out = {}
        with TF.variable_scope('pose', reuse=reuse):
            out['part_pred'] = prediction_layer(dlc_cfg, features, 'part_pred',
                                                dlc_cfg.num_joints)
            if dlc_cfg.location_refinement:
                out['locref'] = prediction_layer(dlc_cfg, features, 'locref_pred',
                                                 dlc_cfg.num_joints * 2)
    
        return out

    def get_net(self, inputs, scmap_inputs):
        net = self.extract_features(inputs, scmap_inputs)
        return self.prediction_layers(net)

    def test(self, inputs, scmap_inputs):
        heads = self.get_net(inputs, scmap_inputs)
        prob = tf.sigmoid(heads['part_pred'])
        return {'part_prob': prob, 'locref': heads['locref']}

    def train(self, batch):
        dlc_cfg = self.dlc_cfg

        if dlc_cfg.deterministic:
            tf.set_random_seed(42)

        heads = self.get_net(batch[Batch.inputs], batch[Batch.scmap_inputs])

        weigh_part_predictions = dlc_cfg.weigh_part_predictions
        part_score_weights = batch[Batch.part_score_weights] if weigh_part_predictions else 1.0

        def add_part_loss(pred_layer):
            return TF.losses.sigmoid_cross_entropy(batch[Batch.part_score_targets],
                                                   heads[pred_layer],
                                                   part_score_weights)

        loss = {}
        loss['part_loss'] = add_part_loss('part_pred')
        total_loss = loss['part_loss']

        if dlc_cfg.location_refinement:
            locref_pred = heads['locref']
            locref_targets = batch[Batch.locref_targets]
            locref_weights = batch[Batch.locref_mask]

            loss_func = losses.huber_loss if dlc_cfg.locref_huber_loss else tf.losses.mean_squared_error
            loss['locref_loss'] = dlc_cfg.locref_loss_weight * loss_func(locref_targets, locref_pred, locref_weights)
            total_loss = total_loss + loss['locref_loss']

    #     # loss['total_loss'] = slim.losses.get_total_loss(add_regularization_losses=params.regularize)
        loss['total_loss'] = total_loss
        return loss



## setup_pose_predict Function

In [0]:
def setup_pose_prediction(dlc_cfg):
    TF.reset_default_graph()
    inputs = TF.placeholder(tf.float32, shape=[dlc_cfg.batch_size   , None, None, 3])
    scoremap_and_locref_channel_multiplier = 3      #JO
    scmap_inputs = TF.placeholder(tf.float32, shape=[dlc_cfg.batch_size   , None, None, dlc_cfg.num_joints * scoremap_and_locref_channel_multiplier])   #JO
    net_heads = PoseNet(dlc_cfg).test(inputs, scmap_inputs)
    outputs = [net_heads['part_prob']]
    if dlc_cfg.location_refinement:
        outputs.append(net_heads['locref'])

    restorer = TF.train.Saver()
    sess = TF.Session()
    sess.run(TF.global_variables_initializer())
    sess.run(TF.local_variables_initializer())

    # Restore variables from disk.
    restorer.restore(sess, dlc_cfg.init_weights)

    return sess, inputs, scmap_inputs, outputs

## Evaluate_Network Function

In [0]:
# Adapted from DLC Evaluate_Network
# deeplabcut.evaluate_network(path_config_file,plotting=True) #debug
# def evaluate_network(config,Shuffles=[1],plotting = None,show_errors = True,comparisonbodyparts="all",gputouse=None): #debug 

%cd /content/cloned-DLC-repo

import numpy as np
from math import floor, ceil    
import pandas as pd
from skimage import io
import skimage.color
import tensorflow as tf
from tqdm import tqdm
from scipy.misc import imresize
import time     #debug
import cv2


from deeplabcut.utils import auxiliaryfunctions 
from deeplabcut.pose_estimation_tensorflow.evaluate import pairwisedistances, make_results_file
from deeplabcut.pose_estimation_tensorflow.nnet import predict as ptf_predict
from deeplabcut.pose_estimation_tensorflow.config import load_config
from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import data_to_input

###
# Parameters from function
###

model_location = path_extension
Shuffles=[1]
comparisonbodyparts="all"
show_errors = True
gputouse=None
jo_interm_sup = False   

###
# Function
###

if 'TF_CUDNN_USE_AUTOTUNE' in os.environ:
    del os.environ['TF_CUDNN_USE_AUTOTUNE'] #was potentially set during training

vers = (tf.__version__).split('.')
if int(vers[0])==1 and int(vers[1])>12:
    TF=tf.compat.v1
else:
    TF=tf

TF.reset_default_graph()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #

start_path=os.getcwd()
# Read file path for pose_config file. >> pass it on
cfg = auxiliaryfunctions.read_config(path_config_file)
if gputouse is not None: #gpu selectinon
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gputouse)

# Loading human annotatated data
trainingsetfolder=auxiliaryfunctions.GetTrainingSetFolder(cfg)
Data=pd.read_hdf(os.path.join(cfg["project_path"],str(trainingsetfolder),'CollectedData_' + cfg["scorer"] + '.h5'),'df_with_missing')
# Get list of body parts to evaluate network for
comparisonbodyparts=auxiliaryfunctions.IntersectionofBodyPartsandOnesGivenbyUser(cfg,comparisonbodyparts)
# Make folder for evaluation
auxiliaryfunctions.attempttomakefolder(str(cfg["project_path"]+"/evaluation-results/"))
for shuffle in Shuffles:
    for trainFraction in cfg["TrainingFraction"]:
        ##################################################
        # Load and setup CNN part detector
        ##################################################
        datafn,metadatafn=auxiliaryfunctions.GetDataandMetaDataFilenames(trainingsetfolder,trainFraction,shuffle,cfg)
        modelfolder=os.path.join(cfg["project_path"],str(GetModelFolder(trainFraction,shuffle,cfg)))    #JO Function
        path_train_config = Path(modelfolder) / 'train' / 'pose_cfg_colab.yaml'     #debug, should modify to test config
        # Load dataset meta data
        data, trainIndices, testIndices, trainFraction=auxiliaryfunctions.LoadMetadata(os.path.join(cfg["project_path"],metadatafn))  #wrong, PSE, <- whas going on here with indices 

        try:
            dlc_cfg = load_config(str(path_train_config))
        except FileNotFoundError:
            raise FileNotFoundError("It seems the model for shuffle %s and trainFraction %s does not exist."%(shuffle,trainFraction))

        #change batch size, if it was edited during analysis!
        dlc_cfg['batch_size']=1 #in case this was edited for analysis.
        #Create folder structure to store results.
        evaluationfolder=os.path.join(cfg["project_path"],str(auxiliaryfunctions.GetEvaluationFolder(trainFraction,shuffle,cfg)),model_version)
        auxiliaryfunctions.attempttomakefolder(evaluationfolder,recursive=True)
        #path_test_config = modelfolder / 'test' / 'pose_cfg_colab.yaml'

        # Check which snapshots are available and sort them by # iterations
        Snapshots = np.array([fn.split('.')[0]for fn in os.listdir(os.path.join(str(model_location), ''))if "index" in fn])
        try: #check if any were found?
          Snapshots[0]
        except IndexError:
          raise FileNotFoundError("Snapshots not found! It seems the dataset for shuffle %s and trainFraction %s is not trained.\nPlease train it before evaluating.\nUse the function 'train_network' to do so."%(shuffle,trainFraction))

        increasing_indices = np.argsort([int(m.split('-')[1]) for m in Snapshots])
        Snapshots = Snapshots[increasing_indices]

        if cfg["snapshotindex"] == -1:
            snapindices = [eval_snap_index]
        elif cfg["snapshotindex"] == "all":
            snapindices = range(len(Snapshots))
        elif cfg["snapshotindex"]<len(Snapshots):
            snapindices=[cfg["snapshotindex"]]
        else:
            print("Invalid choice, only -1 (last), any integer up to last, or all (as string)!")

        final_result=[]
        ##################################################
        # Compute predictions over images
        ##################################################
        for snapindex in snapindices:
            dlc_cfg['init_weights'] = os.path.join(str(model_location),Snapshots[snapindex]) #setting weights to corresponding snapshot.
            trainingsiterations = (dlc_cfg['init_weights'].split(os.sep)[-1]).split('-')[-1] #read how many training siterations that corresponds to.

            #name for deeplabcut net (based on its parameters)
            DLCscorer = GetScorerName(cfg,shuffle,trainFraction,trainingsiterations)     #JO Function
            print("Running ", DLCscorer, " with # of trainingiterations:", trainingsiterations)
            resultsfilename=os.path.join(str(evaluationfolder),DLCscorer + '-' + Snapshots[snapindex]+  '.h5')
            try:
                DataMachine = pd.read_hdf(resultsfilename,'df_with_missing')
                print("This net has already been evaluated!")
            except FileNotFoundError:
                # Specifying state of model (snapshot / training state)
                sess, inputs, scmap_inputs, outputs = setup_pose_prediction(dlc_cfg)

                #JO cobble analysis helper code
                Numimages = len(Data.index)
                start_index = 0
                # list_img_wo_scoremaps = []
                if 'PredicteData' not in locals():
                  PredicteData = np.zeros((Numimages,3 * len(dlc_cfg['all_joints_names'])))
                # else:
                #   hold = 0
                #   hold_index = 0
                #   for j in range(Numimages+1):
                #     if np.sum(PredicteData[j]) == 0:
                #       if j-hold_index>1:
                #         hold = 0
                #       hold_index = j
                #       hold += 1
                #       if hold >=7:
                #         start_index = j - 8
                #         print('Some of PredicteData calculated. Starting at image index',start_index)
                #         break
                # if 'images_without_scmaps' not in locals():
                #   images_without_scmaps = 0
                #   print('images_without_scmaps will only count from this runs start index')
                # else:
                #   print('Some images_without_scmaps already found ({})'.format(images_without_scmaps))

                print("Analyzing data...")
                for imageindex, imagename in tqdm(enumerate(Data.index)):
                    # continue #debug
                    if imageindex < start_index:  
                      continue

                    save_name = str(dlc_cfg.project_path / Path(imagename).parents[0] / Path(imagename).stem)
                    try:
                      scmaps_loaded = np.load(save_name + '.npz', mmap_mode=None, allow_pickle=False, fix_imports=False)  
                    except FileNotFoundError:
                      # print('Eval_Img_Loading')
                      save_name = str(dlc_cfg.project_path / Path(imagename).parents[0] / ('eval_' + Path(imagename).stem))
                      scmaps_loaded = np.load(save_name + '.npz', mmap_mode=None, allow_pickle=False, fix_imports=False)
                      
                      # list_img_wo_scoremaps.append(imagename)
                      # images_without_scmaps += 1
                      # print('\nImage {} has no scmaps. Total now: {}'.format(Path(imagename).parents[0].stem + "/" + Path(imagename).stem,images_without_scmaps))
                      # continue
                    
                    # t0 = time.time()  #debug
                    image = io.imread(os.path.join(cfg['project_path'],imagename),mode='RGB')
                    image = skimage.color.gray2rgb(image)
                    if image.shape[0]==1080:
                      scale = 0.5334903964194936
                    elif image.shape[0]==1520:
                      # continue #debug
                      scale = 0.7579494437963867/2 
                    else:
                      raise ValueError("Image Dimensions are not of a dimension which were expected (some hardcoding involved)")                   
                    img = imresize(image, scale)
                    # if scale == 0.7579494437963867:
                    #   img = img[:,1:,:]
                    image_batch = data_to_input(img)

                    locref = scmaps_loaded['locref']
                    scmap = scmaps_loaded['scmap']
                    scmaps_concat = np.concatenate((locref[:,:,:,0],locref[:,:,:,1],scmap),axis=2)
                    scmaps_concat = cv2.resize(scmaps_concat, dsize=(256, 144), interpolation=cv2.INTER_CUBIC)
                    scmap_batch = data_to_input(scmaps_concat)

                    #PSE, #cobble, temp, TODO, wrong
                    # dim0_increase_needed = ceil((floor((img.shape[0]-1)/2)+1)/2)-scmaps_concat.shape[0]
                    # dim1_increase_needed = ceil((floor((img.shape[1]-1)/2)+1)/2)-scmaps_concat.shape[1]
                    # # print('Debug: Img shape: {}\tScoremap shape: {}\tdim0: {}\tdim1: {}\n'.format(img.shape,scmaps_concat.shape,dim0_increase_needed,dim1_increase_needed))
                    # scmap_padded = np.pad(scmaps_concat,[[floor(dim0_increase_needed/2),dim0_increase_needed-floor(dim0_increase_needed/2)],[floor(dim1_increase_needed/2),dim1_increase_needed-floor(dim1_increase_needed/2)],[0,0]],mode='constant')                    
                    # scmap_batch = data_to_input(scmap_padded)

                    # t1 = time.time()
                    # print('Data load and process took: ',t1-t0) #debug

                    # Compute prediction with the CNN
                    outputs_np = sess.run(outputs, feed_dict={inputs: image_batch,scmap_inputs: scmap_batch})
                    # t2 = time.time()
                    # print('Session run took: ',t2-t1) #debug
                    scmap, locref = ptf_predict.extract_cnn_output(outputs_np, dlc_cfg)

                    # Extract maximum scoring location from the heatmap, assume 1 person
                    if eval_full_scale:
                      scale_multiplier = 1
                    else:
                      scale_multiplier = 2

                    pose = ptf_predict.argmax_pose_predict(scmap, locref, scale_multiplier*(1/scale))
                    # t3 = time.time()
                    # print('Pose Extraction took: ',t3-t2) #debug
                    PredicteData[imageindex, :] = pose.flatten()  # NOTE: thereby     cfg_test['all_joints_names'] should be same order as bodyparts!

                    if imageindex % 500 == 0 and imageindex != 0:
                      print('Saving data on index',imageindex)
                      progress_save_name = evaluationfolder + '/{}_'.format(model_version)+str(trainingsiterations)+'_PredicteData-index'+str(imageindex)
                      np.save(progress_save_name,PredicteData, allow_pickle=False, fix_imports=False)
                sess.close() #closes the current tf session

                index = pd.MultiIndex.from_product( 
                    [[DLCscorer], dlc_cfg['all_joints_names'], ['x', 'y', 'likelihood']],
                    names=['scorer', 'bodyparts', 'coords'])

                # Saving results
                DataMachine = pd.DataFrame(PredicteData, columns=index, index=Data.index.values)
                DataMachine.to_hdf(resultsfilename,'df_with_missing',format='table',mode='w')

                print("Done and results stored for snapshot: ", Snapshots[snapindex])
                DataCombined = pd.concat([Data.T, DataMachine.T], axis=0).T
                RMSE,RMSEpcutoff = pairwisedistances(DataCombined, cfg["scorer"], DLCscorer,cfg["pcutoff"],comparisonbodyparts)
                testerror = np.nanmean(RMSE.iloc[testIndices].values.flatten())
                trainerror = np.nanmean(RMSE.iloc[trainIndices].values.flatten())
                testerrorpcutoff = np.nanmean(RMSEpcutoff.iloc[testIndices].values.flatten())
                trainerrorpcutoff = np.nanmean(RMSEpcutoff.iloc[trainIndices].values.flatten())
                results = [trainingsiterations,int(100 * trainFraction),shuffle,np.round(trainerror,2),np.round(testerror,2),cfg["pcutoff"],np.round(trainerrorpcutoff,2), np.round(testerrorpcutoff,2)]
                final_result.append(results)

                if show_errors == True:
                        print("Results for",trainingsiterations," training iterations:", int(100 * trainFraction), shuffle, "train error:",np.round(trainerror,2), "pixels. Test error:", np.round(testerror,2)," pixels.")
                        print("With pcutoff of", cfg["pcutoff"]," train error:",np.round(trainerrorpcutoff,2), "pixels. Test error:", np.round(testerrorpcutoff,2), "pixels")
                        print("Thereby, the errors are given by the average distances between the labels by DLC and the scorer.")


                if plotting == True:
                    print("Plotting...")
                    colors = visualization.get_cmap(len(comparisonbodyparts),name=cfg['colormap'])

                    foldername=os.path.join(str(evaluationfolder),'LabeledImages_' + DLCscorer + '_' + Snapshots[snapindex])
                    auxiliaryfunctions.attempttomakefolder(foldername)
                    NumFrames=np.size(DataCombined.index)
                    for ind in np.arange(NumFrames):
                        visualization.PlottingandSaveLabeledFrame(DataCombined,ind,trainIndices,cfg,colors,comparisonbodyparts,DLCscorer,foldername)

                TF.reset_default_graph()
                #print(final_result)
        make_results_file(final_result,evaluationfolder,DLCscorer)
        print("The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.")
        print("If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.\nUse the function 'analyze_video' to make predictions on new videos.")
        print("Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)")

# returning to intial folder
os.chdir(str(start_path))


/content/cloned-DLC-repo
/content/drive/Shared drives/Final Year Project/Datasets/Cheetah-AnChi-2019-04-02/evaluation-results/  already exists!
Running  DLC_Extension_resnet50_CheetahApr2shuffle1_250000  with # of trainingiterations: 250000
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Please use `layer.__call__` method instead.


INFO:tensorflow:Restoring parameters from /content/drive/Shared drives/Final Year Project/Datasets/Cheetah-AnChi-2019-04-02/extension-models/iteration-4/CheetahApr2-trainset95shuffle1/train/FsIsTripleHourglass/FS_IS_THG_snapshot-250000


INFO:tensorflow:Restoring parameters from /content/drive/Shared drives/Final Year Project/Datasets/Cheetah-AnChi-2019-04-02/extension-models/iteration-4/CheetahApr2-trainset95shuffle1/train/FsIsTripleHourglass/FS_IS_THG_snapshot-250000
0it [00:00, ?it/s]

Analyzing data...


...

2501it [1:12:27,  1.25it/s]

Saving data on index 2500


2507it [1:12:31,  1.74s/it]


Done and results stored for snapshot:  FS_IS_THG_snapshot-250000
Results for 250000  training iterations: 95 1 train error: 147.06 pixels. Test error: 153.39  pixels.
With pcutoff of 0.5  train error: nan pixels. Test error: nan pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:235: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:236: RuntimeWarning: Mean of empty slice
